In [ ]:
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import subprocess
import json
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import random
import time
import itertools
import os
import random
import h5py
from openTSNE import TSNE
import matplotlib.colors as mcolors
from scipy.cluster.vq import kmeans, vq
import matplotlib.cm as cm
import sqlalchemy
import pyodbc
import urllib
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pyarrow.parquet as pq
import datetime

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
parquet_file = pq.ParquetFile('file')
num_rows = parquet_file.metadata.num_rows
num_subparts = 100
batch_size = num_rows // num_subparts

In [ ]:
df_chunks = []
for i in parquet_file.iter_batches(batch_size=batch_size, use_pandas_metadata=True):
    df_chunks.append(i.to_pandas())

In [ ]:
df = pd.concat(df_chunks, axis=0)

In [ ]:
df['day_str'] = df.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
# temp = pd.read_parquet('total_keywords_with_embeddings.parquet')
# temp = temp.drop('repere', axis=1)
# df = df.drop('cluster', axis=1)
# df = pd.concat([df, temp], axis=0)

In [ ]:
# cluster_df = pd.read_parquet('cluster_df.parquet')

In [ ]:
# keywords_df = pd.read_parquet('all_keywords.parquet')

In [ ]:
len(keywords_df)

In [ ]:
print('---GLOBAL STATS---')
print()
print('oldest date: ' + str(df.date.min()))
print('latest date: ' + str(df.date.max()))
print()
print('number of articles: ' + str(len(df)))
print('number of unique articles: ' + str(len(list(set(df.content.values.tolist())))))
print('number of unique tickers: ' + str(len(df.ticker.unique())))
print()
print('min len of articles: ' + str(df.content.str.len().min()) + ' characters')
print('max len of articles: ' + str(df.content.str.len().max()) + ' characters')
print('mean len of articles: ' + str(df.content.str.len().mean()) + ' characters')

In [ ]:
print('---KEYWORDS STATS---')
print()
print('total number of keywords: ' + str(sum([len(row) for row in df['keywords']])))
print('number of unique keywords: ' + str(len(list(set([x for sublist in df.keywords.values.tolist() for x in sublist])))))
print('mean number of keywords per article: ' + str(df['keywords'].apply(lambda x: len(x)).mean()))
print()
print('top 10 most appearing keywords: \n' + str(pd.DataFrame([x for sublist in df.keywords.values.tolist() for x in sublist]).value_counts().head(5)))

In [ ]:
unwrapped_tickers = []
unwrapped_keywords = []
unwrapped_embeddings = []

for index, row in df.iterrows():
    ticker = row['ticker']
    keywords = row['keywords']
    embeddings = row['keywords_embeddings']
    
    for x,_ in enumerate(keywords):
        unwrapped_tickers.append(ticker)
        unwrapped_keywords.append(keywords[x])
        unwrapped_embeddings.append(embeddings[x])

result = {
    'ticker': unwrapped_tickers,
    'keyword': unwrapped_keywords,
    'embedding': unwrapped_embeddings
}

keywords_df = pd.DataFrame(result)

In [ ]:
keywords_df.keyword = keywords_df.keyword.str.lower()

In [ ]:
len(keywords_df)

In [ ]:
keywords_df = keywords_df.drop_duplicates(subset='keyword')

In [ ]:
len(keywords_df)

In [ ]:
data = np.array(keywords_df['embedding'].tolist())

# Specify the number of clusters (k)
k = 1000

# Perform K-means clustering
centroids, distortion = kmeans(data, k)

# Assign data points to clusters
labels, _ = vq(data, centroids)

# Add the cluster labels to the DataFrame
keywords_df['cluster'] = labels

In [ ]:
# keywords_df.to_parquet('all_keywords.parquet')

In [ ]:
keywords_df

In [ ]:
df['cluster'] = df['keywords'].apply(lambda x: [keywords_df.loc[keywords_df['keyword'] == y, 'cluster'].values[0] for y in x])

In [ ]:
# df.to_parquet('full_merged_df.parquet')

In [ ]:
cluster_df = df.explode('cluster')[['date', 'cluster']]

In [ ]:
cluster_df['value'] = 1
cluster_df.date = cluster_df.date.dt.strftime('%Y-%m-%d')
cluster_df.date = pd.to_datetime(cluster_df.date)
cluster_df = cluster_df.set_index('date')

In [ ]:
cluster_df['value'] = cluster_df.groupby([cluster_df.index, 'cluster'])['value'].transform('sum')
cluster_df = cluster_df.reset_index()
cluster_df = cluster_df.groupby('cluster').apply(lambda x: x.drop_duplicates(subset='date'))
cluster_df = cluster_df.reset_index(drop=True)
cluster_df = cluster_df.set_index('date')

In [ ]:
cluster_df = cluster_df.sort_index()

In [ ]:
cluster_df.value = cluster_df.groupby('cluster').value.cumsum()

In [ ]:
plt.figure(figsize=(15,9))
cluster_df.groupby('cluster').value.plot()
plt.show()

In [ ]:
cluster_df = cluster_df.groupby('cluster').filter(lambda x: len(x) >= 2)

In [ ]:
cluster_df = cluster_df.groupby('cluster').apply(lambda x: x.resample('D').ffill())

In [ ]:
cluster_df = cluster_df.droplevel('cluster')

In [ ]:
def calculate_slope(df):
    df = df.reset_index()
    y = df['value'].values
    df['slope'] = np.gradient(y, df.index)
    df = df.set_index('date')
    return df

cluster_df = cluster_df.groupby('cluster').apply(calculate_slope)

def calculate_slope_gap(df):
    df['slope_gap'] = np.log(df.slope) - np.log(df.shift(1).slope)
    return df

cluster_df = cluster_df.groupby('cluster').apply(calculate_slope_gap)

def calculate_slope_diff(df):
    df['slope_diff'] = df.slope.diff()
    return df

cluster_df = cluster_df.groupby('cluster').apply(calculate_slope_diff)

def calculate_rolling_slope(df):
    df['rolling_slope'] = df.slope.rolling(15).mean()
    return df

# Assuming you have already created and populated the 'cluster' column in x_df
cluster_df = cluster_df.groupby('cluster').apply(calculate_rolling_slope)


In [ ]:
len(df)

In [ ]:
plt.figure(figsize=(15,9))
cluster_df.groupby('cluster').slope.plot()

In [ ]:
# cluster_df.to_parquet('cluster_df.parquet')

In [ ]:
tickers = df.ticker.unique().tolist()

In [ ]:
serv_name_smartbeta = \
    """DRIVER={SQL Server};SERVER=sqlsmartbetaprod\\smartbetaprod;
        DATABASE=SMARTBETA_PROD;Trusted_Connection='Yes''"""
smartbeta = pyodbc.connect(serv_name_smartbeta)

# smartbeta server sqlalchemy connection
quote_smartbeta = \
    urllib.parse.quote_plus(serv_name_smartbeta)
sqlalch_conn = \
    r'mssql+pyodbc:///?odbc_connect={}'\
    .format(quote_smartbeta)
engine = sqlalchemy.create_engine(sqlalch_conn)
conn = smartbeta.cursor()

ticker_for_req = [' '.join([x.split('@')[0] ,x.split('@')[1].replace('GR', 'GY').replace('SW', 'SE').replace('SM', 'SQ')]) for x in tickers]

ticker_str = '(' + ', '.join([f"'{ticker}'" for ticker in ticker_for_req]) + ')'

start_date = '2018-01-01'
query = f"""select TT.fsym_id,
       TT.date,
       TT.tot_ret_euro,
       TT.Bloom_Nego   
from
(
SELECT  *
FROM [SMARTBETA_PROD].[dbo].[data_Tot_Ret_daily] as TR
join
equity_info_codes() EIC
on EIC.fsym_regional_id = TR.fsym_id
where TR.date >= '2018-01-01' 
and EIC.Bloom_Nego in {ticker_str}) as TT"""

query_cac = f"""SELECT II.Code_instrument,
        II.Code_Bloom,
                                   RTRIM(II.Devise) as Devise,
                                   HI.Cours_Local,
                                   HI.date
                            FROM [SMARTBETA_PROD].[dbo].[instr_Index] II
                            JOIN [SMARTBETA_PROD].[dbo].[histo_instruments] HI
                            ON II.Code_instrument = HI.Code_instrument
                            WHERE II.Code_Bloom = 'SXXP Index'
                            AND HI.date >= '2018-01-01'"""

sxxp_returns = pd.read_sql_query(query_cac, engine)
daily_returns = pd.read_sql_query(query, engine)
daily_returns.date = pd.to_datetime(daily_returns.date)

pivot_df = daily_returns.pivot_table(index='date', columns='Bloom_Nego', values='tot_ret_euro')
pivot_df.columns = ['@'.join(x.split(' ')) for x in pivot_df.columns]

In [ ]:
pivot_df.iloc[0] = 0

In [ ]:
sxxp_returns.date = pd.to_datetime(sxxp_returns.date)
sxxp_returns = sxxp_returns.set_index('date')

In [ ]:
missing_tickers = [x for x in ticker_for_req if '@'.join(x.split(' ')) not in pivot_df.columns]
print(len(missing_tickers))
missing_tickers

In [ ]:
df.ticker = df.ticker.apply(lambda x: '@'.join([x.split('@')[0] ,x.split('@')[1].replace('GR', 'GY').replace('SW', 'SE').replace('SM', 'SQ')]))

In [ ]:
cluster_df[cluster_df.slope >=12].cluster.unique()

In [ ]:
def display_cluster_keywords(cluster):
    print(keywords_df[keywords_df.cluster == cluster].keyword.head(10))

In [ ]:
def trending_clusters_overview(cluster):
    selected_cluster_df = df[df.cluster.apply(lambda x: cluster in x)]
    filtered_df = cluster_df[cluster_df.cluster == cluster]
    selected_cluster_df = selected_cluster_df[selected_cluster_df.date > '2017-01-01']
    selected_cluster_df['overall'] = 1


    monthly_article_counts = selected_cluster_df.pivot_table(index='year_month', columns='ticker', values='content', aggfunc='count', fill_value=0)

    weekly_article_counts = selected_cluster_df.pivot_table(index='year_week', columns='ticker', values='content', aggfunc='count', fill_value=0)

    daily_article_counts = selected_cluster_df.pivot_table(index='date', columns='ticker', values='content', aggfunc='count', fill_value=0)


    monthly_article_counts[[ticker for ticker in pivot_df.columns if ticker not in monthly_article_counts]] = 0
    weekly_article_counts[[ticker for ticker in pivot_df.columns if ticker not in weekly_article_counts]] = 0
    daily_article_counts[[ticker for ticker in pivot_df.columns if ticker not in daily_article_counts]] = 0


    monthly_article_counts = monthly_article_counts.shift(1).fillna(0.0)
    weekly_article_counts = weekly_article_counts.shift(1).fillna(0.0)
    daily_article_counts = daily_article_counts.shift(1).fillna(0.0)


    monthly_article_counts = monthly_article_counts[[x for x in monthly_article_counts.columns if x in pivot_df.columns]]

    weekly_article_counts = weekly_article_counts[[x for x in weekly_article_counts.columns if x in pivot_df.columns]]

    daily_article_counts = daily_article_counts[[x for x in daily_article_counts.columns if x in pivot_df.columns]]


    monthly_weights = monthly_article_counts.div(monthly_article_counts.sum(axis=1), axis=0)

    weekly_weights = weekly_article_counts.div(weekly_article_counts.sum(axis=1), axis=0)

    daily_weights = daily_article_counts.div(daily_article_counts.sum(axis=1), axis=0)


    cluster_tickers_returns = pivot_df[[x for x in monthly_weights.columns]]


    monthly_weights.index = pd.to_datetime(monthly_weights.index)

    weekly_weights.index = pd.to_datetime(weekly_weights.index + '-0', format='%Y-%U-%w')

    daily_weights = daily_weights.resample('D').sum()
    daily_weights.index = pd.to_datetime(daily_weights.index.strftime('%Y-%m-%d'))

    cluster_tickers_returns.index = pd.to_datetime(cluster_tickers_returns.index)


    weekly_weights = weekly_weights[~weekly_weights.index.duplicated(keep='first')]


    monthly_weights = monthly_weights.reindex(cluster_tickers_returns.index, method='ffill')

    weekly_weights = weekly_weights.reindex(cluster_tickers_returns.index, method='ffill')

    daily_weights = daily_weights.reindex(cluster_tickers_returns.index, method='ffill')


    monthly_rebalanced_weighted_returns = cluster_tickers_returns * monthly_weights

    weekly_rebalanced_weighted_returns = cluster_tickers_returns * weekly_weights

    daily_rebalanced_weighted_returns = cluster_tickers_returns * daily_weights


    fig, ax1 = plt.subplots()
    filtered_df.slope.plot(ax=ax1, color='lightgray')
    filtered_df.rolling_slope.plot(ax=ax1, color='lightblue')
    ax1.legend(loc='upper left')

    # ax2 = ax1.twinx()
    # filtered_df.value.plot(ax=ax2, color='r', label='cluster cumulative appearance')
    # ax2.legend(loc='lower right')

    ax3 = ax1.twinx()
    (monthly_rebalanced_weighted_returns.sum(axis=1).cumsum()/sxxp_returns.Cours_Local).plot(ax=ax3, figsize=(18, 12), label='custom monthly rebalanced portfolio returns / index returns', color='orange')
    (weekly_rebalanced_weighted_returns.sum(axis=1).cumsum()/sxxp_returns.Cours_Local).plot(ax=ax3, label='custom weekly rebalanced portfolio returns / index returns', color='purple')
    (daily_rebalanced_weighted_returns.sum(axis=1).cumsum()/sxxp_returns.Cours_Local).plot(ax=ax3, label='custom daily rebalanced portfolio returns / index returns', color='red')
    plt.legend(loc='upper right')

    plt.show()


    (monthly_weights.max(axis=1)/monthly_weights.sum(axis=1)).plot(figsize=(15,9))
    (weekly_weights.max(axis=1)/weekly_weights.sum(axis=1)).plot(figsize=(15,9))
    (daily_weights.max(axis=1)/daily_weights.sum(axis=1)).plot(figsize=(15,9))

    plt.show()

In [ ]:
keywords_df[keywords_df.keyword.str.contains('vaccine')]

In [ ]:
int_dropdown = widgets.Dropdown(options=cluster_df[cluster_df.slope >=12].cluster.unique().tolist())

# Use the interact function with the Dropdown widget
interact(trending_clusters_overview, cluster=int_dropdown)

interact(display_cluster_keywords, cluster=int_dropdown)

In [ ]:
def analyse_cluster_content(cluster, start_date, snapshot_date):

    start_date = start_date.strftime('%Y-%m-%d')
    snapshot_date = snapshot_date.strftime('%Y-%m-%d')

    selected_cluster_df = df[df.cluster.apply(lambda x: cluster in x)]
    selected_cluster_df = selected_cluster_df[selected_cluster_df.date >= start_date]
    selected_cluster_df['overall'] = 1


    monthly_article_counts = selected_cluster_df.pivot_table(index='year_month', columns='ticker', values='content', aggfunc='count', fill_value=0)

    weekly_article_counts = selected_cluster_df.pivot_table(index='year_week', columns='ticker', values='content', aggfunc='count', fill_value=0)

    daily_article_counts = selected_cluster_df.pivot_table(index='date', columns='ticker', values='content', aggfunc='count', fill_value=0)


    monthly_article_counts[[ticker for ticker in pivot_df.columns if ticker not in monthly_article_counts]] = 0
    weekly_article_counts[[ticker for ticker in pivot_df.columns if ticker not in weekly_article_counts]] = 0
    daily_article_counts[[ticker for ticker in pivot_df.columns if ticker not in daily_article_counts]] = 0


    monthly_article_counts = monthly_article_counts.shift(1).fillna(0.0)
    weekly_article_counts = weekly_article_counts.shift(1).fillna(0.0)
    daily_article_counts = daily_article_counts.shift(1).fillna(0.0)


    monthly_article_counts = monthly_article_counts[[x for x in monthly_article_counts.columns if x in pivot_df.columns]]

    weekly_article_counts = weekly_article_counts[[x for x in weekly_article_counts.columns if x in pivot_df.columns]]

    daily_article_counts = daily_article_counts[[x for x in daily_article_counts.columns if x in pivot_df.columns]]


    monthly_weights = monthly_article_counts.div(monthly_article_counts.sum(axis=1), axis=0)

    weekly_weights = weekly_article_counts.div(weekly_article_counts.sum(axis=1), axis=0)

    daily_weights = daily_article_counts.div(daily_article_counts.sum(axis=1), axis=0)


    cluster_tickers_returns = pivot_df[[x for x in monthly_weights.columns]]


    monthly_weights.index = pd.to_datetime(monthly_weights.index)

    weekly_weights.index = pd.to_datetime(weekly_weights.index + '-0', format='%Y-%U-%w')

    daily_weights = daily_weights.resample('D').sum()
    daily_weights.index = pd.to_datetime(daily_weights.index.strftime('%Y-%m-%d'))

    cluster_tickers_returns.index = pd.to_datetime(cluster_tickers_returns.index)


    weekly_weights = weekly_weights[~weekly_weights.index.duplicated(keep='first')]


    monthly_weights = monthly_weights.reindex(cluster_tickers_returns.index, method='ffill')

    weekly_weights = weekly_weights.reindex(cluster_tickers_returns.index, method='ffill')

    daily_weights = daily_weights.reindex(cluster_tickers_returns.index, method='ffill')

    print(f'start_date: {start_date} | snapshot_date: {snapshot_date}')
    if snapshot_date in [x.strftime('%Y-%m-%d') for x in daily_weights.index.tolist()] and daily_weights.loc[snapshot_date].nlargest(10).max()>0:
        print('composition:')
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        axs[2].set_title('Daily')
        weights = daily_weights.loc[snapshot_date].nlargest(10)
        axs[2].pie(weights, labels=weights.index, autopct=lambda x: (x/abs(daily_weights.loc[snapshot_date]).sum()).round(2))
        axs[2].axis('equal')
        axs[2].set_ylabel('')

        axs[1].set_title('Weekly')
        weights = weekly_weights.loc[snapshot_date].nlargest(10)
        axs[1].pie(weights, labels=weights.index, autopct=lambda x: (x/abs(weekly_weights.loc[snapshot_date]).sum()).round(2))
        axs[1].axis('equal')
        axs[1].set_ylabel('')

        axs[0].set_title('Monthly')
        weights = monthly_weights.loc[snapshot_date].nlargest(10)
        axs[0].pie(weights, labels=weights.index, autopct=lambda x: (x/abs(monthly_weights.loc[snapshot_date]).sum()).round(2))
        axs[0].axis('equal')
        axs[0].set_ylabel('')

        plt.show()
    else:
        print('weights are NaNs because of shifting/rolliing OR date not found in daily returns index')

In [ ]:
valid_dates = df.day_str.unique()
dt_valid = [datetime.datetime.strptime(x, '%Y-%m-%d').date() for x in valid_dates]

start_date = widgets.DatePicker(
    description='Start Date',
    value=(min(dt_valid)),
    min=(min(dt_valid)),
    max=(max(dt_valid))
    )
snapshot_date = widgets.DatePicker(
    description='End Date',
    value=(min(dt_valid)),
    min=(min(dt_valid)),
    max=(max(dt_valid))
    )

interact(analyse_cluster_content, cluster=widgets.IntSlider(min=0, max=1000), start_date=start_date, snapshot_date=snapshot_date)

# analyse_cluster_content(723, '2018-01-01', '2020-12-11')